In [1]:
import datetime, os, re, requests
import pandas as pd

In [2]:

def mirror(url, dest):
    """Download a file from url and save it locally under dest"""
    if os.path.exists(dest):
        print(f"File {dest} already exists, skipping")
        return

    print(f"Downloading {url} to {dest}")
    # Use a recent chrome user agent to avoid 403 error
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
                             'AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/92.0.4515.159 Safari/537.36'}
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        raise Exception(f"Got status code {resp.status_code} for {url}: {resp.text}")
    data = resp.content
    with open(dest, 'wb') as handler:
        handler.write(data)
    print(f"Wrote {len(data)/1e6} MB to {dest}")

mirror("https://www.epa.gov/system/files/other-files/2023-06/Fenceline%20data%20for%20five%20facilities%202022-2023.zip", "Fenceline_data_for_five_facilities_2022-2023.zip")

File Fenceline_data_for_five_facilities_2022-2023.zip already exists, skipping


In [3]:
# Unzip into Fenceline_data_for_five_facilities directory


In [4]:
clairton_5_3_23 = pd.read_excel("Fenceline_data_for_five_facilities_2022-2023/ANSWER-FUGITIVE-Coke Enc2_Test Results_USSteel_Clairton_5-3-23.xlsx", sheet_name="Sample Results", skiprows=range(1,6))

def sanitize_column_name(name):
    name = re.sub(r'\s+', ' ', name)
    name = name.replace("(at least five decimal places)", "")
    name = name.replace("(Select from dropdown list)", "")
    name = name.strip()
    return name

clairton_5_3_23.rename(columns = sanitize_column_name, inplace=True)

clairton_5_3_23

/Users/rsargent/projects/epa-coke-oven-neshap-comments/.venv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Sampling Period Start Date,Sampling Period Start Time,Sampling Period End Date,Sampling Period End Time,Sampler Name,Latitude of Passive Sampler in Decimal Degree,Longitude of Passive Sampler in Decimal Degree,Method,Sample Type,Compound,Sampling Period Concentration (µg/m3),Below method detection limit (BDL)?,Lab Reported Concentration for BDL Sample (µg/m3),Other Data Flag(s),Explanation
0,1/3/2023,08:35:00,1/17/2023,09:14:00,USSCL-PT01-S-20230103,40.310273,-79.884844,Method 325A,Regular Monitor,"1,3-Butadiene",ND,yes,0.609,ND,NaN
1,1/3/2023,08:35:00,1/17/2023,09:14:00,USSCL-PT01-S-20230103,40.310273,-79.884844,Method 325A,Regular Monitor,Benzene,8.66,no,0.192,NaN,NaN
2,1/3/2023,08:35:00,1/17/2023,09:14:00,USSCL-PT01-S-20230103,40.310273,-79.884844,Method 325A,Regular Monitor,Ethylbenzene,ND,yes,0.613,ND,NaN
3,1/3/2023,08:35:00,1/17/2023,09:14:00,USSCL-PT01-S-20230103,40.310273,-79.884844,Method 325A,Regular Monitor,m-/p-Xylenes,ND,yes,0.617,ND,NaN
4,1/3/2023,08:35:00,1/17/2023,09:14:00,USSCL-PT01-S-20230103,40.310273,-79.884844,Method 325A,Regular Monitor,o-Xylene,ND,yes,0.621,ND,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2023-03-27 00:00:00,11:27:00,2023-04-11 00:00:00,09:11:00,USSCL-PT12-S-20230327,40.309223,-79.880625,Method 325A,Regular Monitor,Benzene,10.2,no,0.179,Rc,Recollection analysis
584,2023-03-27 00:00:00,11:27:00,2023-04-11 00:00:00,09:11:00,USSCL-PT12-S-20230327,40.309223,-79.880625,Method 325A,Regular Monitor,Ethylbenzene,ND,yes,1.130,"ND,Rc",Recollection analysis
585,2023-03-27 00:00:00,11:27:00,2023-04-11 00:00:00,09:11:00,USSCL-PT12-S-20230327,40.309223,-79.880625,Method 325A,Regular Monitor,m-/p-Xylenes,ND,yes,1.140,"ND,Rc",Recollection analysis
586,2023-03-27 00:00:00,11:27:00,2023-04-11 00:00:00,09:11:00,USSCL-PT12-S-20230327,40.309223,-79.880625,Method 325A,Regular Monitor,o-Xylene,ND,yes,1.140,"ND,Rc",Recollection analysis


In [5]:
clairton_5_3_23[['Method', 'Sample Type', 'Compound', "Below method detection limit (BDL)?"]].value_counts()

Method       Sample Type      Compound       Below method detection limit (BDL)?
Method 325A  Regular Monitor  Toluene        no                                     84
                              Benzene        no                                     84
                              1,3-Butadiene  yes                                    84
                              Ethylbenzene   yes                                    83
                              o-Xylene       yes                                    82
                              m-/p-Xylenes   yes                                    52
                                             no                                     32
             Duplicate        Benzene        no                                      7
                              1,3-Butadiene  yes                                     7
                              Ethylbenzene   yes                                     7
                              Toluene        no  

In [6]:
clairton_bdl = pd.read_excel("Fenceline_data_for_five_facilities_2022-2023/ANSWER-FUGITIVE-Coke Enc2_Test Results_USSteel_Clairton_BDL.xlsx", sheet_name="Sample Results")

def sanitize_column_name(name):
    name = re.sub(r'\s+', ' ', name)
    name = name.replace("(at least five decimal places)", "")
    name = name.replace("(Select from dropdown list)", "")
    name = name.strip()
    return name

clairton_bdl.rename(columns = sanitize_column_name, inplace=True)

#clairton_bdl['Sampling Period Start Date'] = clairton_bdl["Sampling Period Start Date"].astype(datetime.date)
clairton_bdl

/Users/rsargent/projects/epa-coke-oven-neshap-comments/.venv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Sampling Period Start Date,Sampling Period Start Time,Sampling Period End Date,Sampling Period End Time,Sampler Name,Latitude of Passive Sampler in Decimal Degree,Longitude of Passive Sampler in Decimal Degree,Method,Sample Type,Compound,Sampling Period Concentration (µg/m3),Below method detection limit (BDL)?,Lab Reported Concentration for BDL Sample (µg/m3),Other Data Flag(s),Explanation
0,12/19/2022,08:11:00,1/3/2023,08:35:00,USSCL-PT01-S-20221219,40.310273,-79.884844,Method 325A,Regular Monitor,"1,3-Butadiene",0.572,yes,0.572,ND,NaN
1,12/19/2022,08:11:00,1/3/2023,08:35:00,USSCL-PT01-S-20221219,40.310273,-79.884844,Method 325A,Regular Monitor,Ethylbenzene,0,yes,0.576,ND,NaN
2,2022-10-11 00:00:00,07:22:00,2022-10-25 00:00:00,07:56:00,USSCL-PT01-S-20221011,40.310273,-79.884844,Method 325A,Regular Monitor,o-Xylene,0.61,yes,0.610,ND,NaN
3,12/19/2022,08:16:00,1/3/2023,08:43:00,USSCL-PT02-S-20221219,40.308475,-79.886978,Method 325A,Regular Monitor,"1,3-Butadiene",0,yes,0.571,ND,NaN
4,12/19/2022,08:16:00,1/3/2023,08:43:00,USSCL-PT02-S-20221219,40.308475,-79.886978,Method 325A,Regular Monitor,Ethylbenzene,0,yes,0.576,ND,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4520,NaN,NaN,NaN,NaN,X010923A_Humid Blank #0702,NaN,NaN,TO-15A,Method Blank,1-Bromopropane,NaN,yes,0.176,ND,Blank for samples collected on 01/04/23
4521,NaN,NaN,NaN,NaN,X010923A_Humid Blank #0702,NaN,NaN,TO-15A,Method Blank,1-Octene,NaN,yes,0.161,ND,Blank for samples collected on 01/04/23
4522,NaN,NaN,NaN,NaN,X010923A_Humid Blank #0702,NaN,NaN,TO-15A,Method Blank,n-Octane,NaN,yes,0.163,ND,Blank for samples collected on 01/04/23
4523,NaN,NaN,NaN,NaN,X010923A_Humid Blank #0702,NaN,NaN,TO-15A,Method Blank,Isopropylbenzene,NaN,yes,0.172,ND,Blank for samples collected on 01/04/23


In [7]:
pd.DataFrame(clairton_bdl[['Method', 'Sample Type', 'Compound', "Below method detection limit (BDL)?"]].value_counts())

count
Method      Sample Type     Compound               Below method detection limit (BDL)?       
Method 325A Regular Monitor 1,3-Butadiene          yes                                     72
                            Benzene                no                                      72
                            Toluene                no                                      72
                            Ethylbenzene           yes                                     71
                            o-Xylene               yes                                     67
...                                                                                       ...
TO-15A      Duplicate       1,2,4-Trimethylbenzene yes                                      1
TO-13A      Regular Monitor Benzo(g,h,i)perylene   no                                       1
TO-15A      Duplicate       n-Octane               no                                       1
TO-13A      Regular Monitor Phenanthrene           yes                                      1
TO-15A      Duplicate       Ethylbenzene           no                                       1

[329 rows x 1 columns]

In [8]:
# Show 500 rows of data
pd.set_option('display.max_rows', 500)

# Convert ND to 0
clairton_bdl['Sampling Period Concentration (µg/m3)'] = clairton_bdl['Sampling Period Concentration (µg/m3)'].replace('ND', 0)


clairton_bdl[clairton_bdl['Method'] == 'TO-13A'].sort_values('Sampling Period Concentration (µg/m3)', ascending=False)


,Sampling Period Start Date,Sampling Period Start Time,Sampling Period End Date,Sampling Period End Time,Sampler Name,Latitude of Passive Sampler in Decimal Degree,Longitude of Passive Sampler in Decimal Degree,Method,Sample Type,Compound,Sampling Period Concentration (µg/m3),Below method detection limit (BDL)?,Lab Reported Concentration for BDL Sample (µg/m3),Other Data Flag(s),Explanation
1766,2023-01-03 00:00:00,12:15:00,2023-01-04 00:00:00,12:08:00,PAH04_230103_S,40.304261,-79.876256,TO-13A,Regular Monitor,Naphthalene (naphthene),100.451812,no,1.461117,NaN,NaN
570,2022-10-25 00:00:00,11:06:00,2022-10-26 00:00:00,11:08:00,PAH04_221025_S,40.304261,-79.876256,TO-13A,Regular Monitor,Naphthalene (naphthene),93.518643,no,1.554596,NaN,NaN
54,2022-10-11 00:00:00,09:33:00,2022-10-12 00:00:00,09:31:00,PAH04_221011_S,40.304261,-79.876256,TO-13A,Regular Monitor,Naphthalene (naphthene),80.527707,no,0.214027,NaN,NaN
1119,2022-12-19 00:00:00,10:22:00,2022-12-20 00:00:00,10:28:00,PAH04_221219_S,40.304261,-79.876256,TO-13A,Regular Monitor,Naphthalene (naphthene),61.135231,no,0.716603,NaN,NaN
890,2022-12-06 00:00:00,15:55:00,2022-12-07 00:00:00,13:21:00,PAH04_221206_S,40.304261,-79.876256,TO-13A,Regular Monitor,Naphthalene (naphthene),48.353048,no,0.409338,NaN,NaN
727,2022-11-22 00:00:00,11:19:00,2022-11-23 00:00:00,11:19:00,PAH04_221122_S,40.304261,-79.876256,TO-13A,Regular Monitor,Naphthalene (naphthene),43.447864,no,0.386203,NaN,NaN
641,2022-11-08 00:00:00,16:02:00,2022-11-09 00:00:00,16:02:00,PAH04_221108_S,40.304261,-79.876256,TO-13A,Regular Monitor,Naphthalene (naphthene),39.334886,no,0.801730,NaN,NaN
649,2022-11-08 00:00:00,16:37:00,2022-11-09 00:00:00,16:37:00,PAH02_221108_S,40.303927,-79.871962,TO-13A,Regular Monitor,Naphthalene (naphthene),31.425912,no,0.798118,NaN,NaN
555,2022-10-11 00:00:00,10:13:00,2022-10-12 00:00:00,10:06:00,PAH02_221011_S,40.303927,-79.871962,TO-13A,Regular Monitor,Naphthalene (naphthene),16.080120,no,0.197000,NaN,NaN
574,2022-10-25 00:00:00,11:36:00,2022-10-26 00:00:00,11:39:00,PAH03_221025_S,40.308241,-79.878017,TO-13A,Regular Monitor,Naphthalene (naphthene),11.634986,no,0.197203,NaN,NaN


In [13]:
found_pahs = clairton_bdl[clairton_bdl['Method'] == 'TO-13A'][['Compound', 'Sampling Period Concentration (µg/m3)']].groupby('Compound').max()
found_pahs.to_csv('found_pahs.csv')
found_pahs


,Sampling Period Concentration (µg/m3)
Compound,
Acenaphthene,0.255696
Acenaphthylene,8.173125
Anthracene,0.404090
Benz(a)anthracene,0.068301
Benzo(a)pyrene,0.016041
Benzo(b)fluoranthene,0.017422
"Benzo(g,h,i)perylene",0.008980
Benzo(k)fluoranthene,0.040309
Chrysene,0.053282


In [25]:
# Per https://www.epa.gov/sites/default/files/2016-09/documents/coke-oven-emissions.pdf
coe_ref_concentration = 0.002 # ug/m^3

sample_sums = clairton_bdl[clairton_bdl['Method'] == 'TO-13A'][['Sampler Name', 'Sampling Period Concentration (µg/m3)']].groupby('Sampler Name').sum()
sample_sums['Cancers per million'] = sample_sums['Sampling Period Concentration (µg/m3)'] / coe_ref_concentration
sample_sums['Cancers one in'] = (1000000 / sample_sums['Cancers per million']).round()
sample_sums

,Sampling Period Concentration (µg/m3),Cancers per million,Cancers one in
Sampler Name,,,
PAH01_221011_S,0.524565,262.282263,3813.0
PAH01_221025_S,0.531737,265.868427,3761.0
PAH01_221108_S,1.175652,587.825772,1701.0
PAH01_221122_S,0.376138,188.068861,5317.0
PAH01_221206_S,1.267514,633.757044,1578.0
PAH01_221219_S,0.128533,64.266704,15560.0
PAH01_230103_S,0.000000,0.000000,inf
PAH02_221011_S,16.652651,8326.325541,120.0
PAH02_221025_S,2.870534,1435.267231,697.0


In [26]:
sample_sums_no_napthalene = clairton_bdl[(clairton_bdl['Method'] == 'TO-13A') & (clairton_bdl['Compound'] != 'Naphthalene (naphthene)')][['Sampler Name', 'Sampling Period Concentration (µg/m3)']].groupby('Sampler Name').sum()
sample_sums_no_napthalene['Cancers per million'] = sample_sums_no_napthalene['Sampling Period Concentration (µg/m3)'] / coe_ref_concentration
sample_sums_no_napthalene['Cancers one in'] = (1000000 / sample_sums_no_napthalene['Cancers per million']).round()
sample_sums_no_napthalene

,Sampling Period Concentration (µg/m3),Cancers per million
Sampler Name,,
PAH01_221011_S,0.100344,50.171759
PAH01_221025_S,0.135231,67.615438
PAH01_221108_S,0.045217,22.608684
PAH01_221122_S,0.037275,18.637455
PAH01_221206_S,0.182973,91.486683
PAH01_221219_S,0.008542,4.270889
PAH01_230103_S,0.000000,0.000000
PAH02_221011_S,0.572531,286.265536
PAH02_221025_S,0.136692,68.346059
